In [ ]:
# One can clone the data into colaboratory, but this has to be repeated every time one runs the notebook (takes 1 GB of space)
# This space is NOT in your computer and is just on a server.
!git clone https://github.com/BU-hammerTeam/PyHammer.git


# Alternatively one can save a sub-set of data into their google drive, and connect colaboratory with drive.
# This is NOT mandatory and is only helpfull is someone is more familiar with python and likes to try this.
# You only need to save data once...
# Download data and latest version of code
# to access data follow the link and download
# https://drive.google.com/drive/folders/1IzcCNYb01jyBqiNzRBqmj39yfLBRWP93?usp=sharing
# Save data into drive, in the 'Astro_lab/DATA/' folder 
# you can also add a shortcut on shared with me folders.
# https://support.google.com/drive/answer/2375057?hl=en&co=GENIE.Platform%3DDesktop
# or...
# Right-click on Shared Folders or files.
# 2. Click on Download
# Downloading Status will be shown in the bottom right corner.
# 3. Goto My Drive
# 4. Right-click on the empty area and click on "Upload files" for file or "Upload folder" for the folder.
# 5. Select the location of the downloaded files and click on "Open".

# https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive
# USE THIS TO READ DATA FROM Google DRIVE
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# This isntalls spceutils, a tool/function used to analyze spectral data
!pip install specutils

In [ ]:
# This is a set of functions used for interactive plotting  
!pip install --upgrade plotly

In [ ]:
# function used to read fits files. FITS are esentially text files that store multidimensional tables with specific format that is not read by normal human eyes :)
from astropy.io import fits

# function used to assing units to values, also helpfull to make calculations in SI CGS ets.
from astropy import units as u

#numerical library of functions
import numpy as np

import plotly.express as px #plotly is another -interactive- library for making plots

#spectra utilities
from specutils import Spectrum1D,SpectralRegion
from specutils.analysis import equivalent_width

# functions to access paths in operetional system and other ...
import os.path

# this imports pandas, a tool used for dataframes or 'tables'
import pandas as pd

In [ ]:
# What files do we have?
# colaboratory works under a unix/linux terminal, so we can see what files are stored in our server
# se by adding '!' at the start of the line we can use linux commands like 'ls' or list directory... same as DIR in DOS
# so lets look what templates we downloaded with gic clone pyHAMMER
!ls PyHammer/resources/templates/

In [ ]:
#USE THIS if you called git clone and pyHAMMER
f=fits.open("PyHammer/resources/templates/M3_+0.0_Dwarf.fits")

#USE this for file in G Drive
# f=fits.open("/content/drive/My Drive/Astro_lab/DATA/M3_+0.0_Dwarf.fits")
specdata = f[1].data 
specheader = f[0].header

In [ ]:
# these are the data of the fits table
specdata

In [ ]:
lamb = 10**specdata['loglam'] * u.AA 
flux = specdata['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1') 
spec = Spectrum1D(spectral_axis=lamb, flux=flux) 

#this is the header of the fits table... so information about columns, instrument or other stuff that someone who created the table found it usefull
print('------------------------------ Header ------------------------------')
print(repr(specheader))
print('---------------------------- End Header ----------------------------')

In [ ]:
# define wavelength of some line 
wv_Ca_K = 3933.7 
wv_Ca_H = 3968.5 
wv_Ha = 6562.8 


In [ ]:
# Make a plot with wavelength and flux and plot vertical lines as specific energies
fig=px.line(x=lamb, y=flux,labels={"x": f"Wavelength [{lamb.unit.to_string()}]","y": f"Flux [{flux.unit.to_string()}]"}, width=1000, height=300)
fig.update_layout(
    margin=dict(l=0, r=5, t=0, b=0),
    paper_bgcolor="LightSteelBlue",
)
fig.add_vline(wv_Ca_K)
fig.add_vline(wv_Ca_H)
fig.add_vline(wv_Ha)
fig

In [ ]:
# To avoid writing the same code for every plot.. e.g. we need to plot 40 spectra we define a function that can use 3 inputs to make the plot
# spectral_type: A, B, G, O etc
# sub_spectral_type: 0,1...9
# metalicity: +0.0, -0.5, +0.5   be carefull + is needed 
#  line: possition to draw a vertical line

def plot_spectra(spectral_type,sub_spectral_type,metalicity,line=None):
  fname=f"PyHammer/resources/templates/{spectral_type}{sub_spectral_type}_{metalicity}_Dwarf.fits"
  #USE this for file in G Drive
  # fname=f"/content/drive/My Drive/Astro_lab/DATA/{spectral_type}{sub_spectral_type}_{metalicity}_Dwarf.fits"
  if os.path.isfile(fname):
    f=fits.open(fname)
    specdata = f[1].data 
    lamb = 10**specdata['loglam'] * u.AA 
    flux = specdata['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1') 
    fig=px.line(x=lamb, y=flux,labels={"x": f"Wavelength [{lamb.unit.to_string()}]","y": f"Flux [{flux.unit.to_string()}]"}, width=1000, height=300)
    fig.update_layout(
        margin=dict(l=0, r=5, t=0, b=0),
        paper_bgcolor="LightSteelBlue",
    )
    if line is not None:
      fig.add_vline(line)
    return fig
  else:
    raise ValueError(f"The template {fname} does't exist")

In [ ]:
# run function with 4 inputs
# see the lines, zoom in and think about the continuum.. where would you drow it?

plot_spectra("F","8","-1.5",wv_Ha)


# uncomment and try next one... after commenting out above... 
# plot_spectra("G","2","+0.0",8542)


In [ ]:
# Try to plot a B type spectrum...
plot_spectra("B","0","",wv_Ha)

# An error will occur, since the plot_spectra function is not ideal.
# It uses specific file format.

In [ ]:
#Some files are stored with different names, e.g. B0.fits	
# This is NOT asked for the exersise, but if anyone likes to try and see all spectral types you can play with this cell
# So here is a working example, perhaps you can try and make a function that works with B type stars

f=fits.open("PyHammer/resources/templates/B0.fits")
specdata = f[1].data 
specheader = f[0].header
lamb = 10**specdata['loglam'] * u.AA 
flux = specdata['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1') 
spec = Spectrum1D(spectral_axis=lamb, flux=flux) 


fig=px.line(x=lamb, y=flux,labels={"x": f"Wavelength [{lamb.unit.to_string()}]","y": f"Flux [{flux.unit.to_string()}]"}, width=1000, height=300)
fig.update_layout(
    margin=dict(l=0, r=5, t=0, b=0),
    paper_bgcolor="LightSteelBlue",
)
fig.add_vline(wv_Ca_K)
fig.add_vline(wv_Ca_H)
fig.add_vline(wv_Ha)
fig

In [ ]:
# Now we define a function that estimates the equivalent width.
# We need the input file defined by the first 3 parameters
# the next 4 inputs are the left1-left2, right1-right2 the ranges under which continuum is estimated
# After continuum is estimated the spectrum is subtracted by the continuum (linear function) and integrated
# Area below the continuum is negative and above positive.
# By integrating between left2-right1 we get the line strength which is then translated to equivalent streangth
# This is done by 'equivalent_width' function imported by 'specutils.analysis' package
# The final parameter... 'line' is only used for ploting purposes i.e. plotting a rectangular with area equal to the equivalent width around the input value.
# Changing this last value will not change the calculation, only the plot


def estimate_eq(spectral_type,sub_spectral_type,metalicity,x_left1,x_left2,x_right1,x_right2,line,make_plots=True):
  fname=f"PyHammer/resources/templates/{spectral_type}{sub_spectral_type}_{metalicity}_Dwarf.fits"
  #USE this for file in G Drive
  # fname=f"/content/drive/My Drive/Astro_lab/DATA/{spectral_type}{sub_spectral_type}_{metalicity}_Dwarf.fits"
  if os.path.isfile(fname):
    f=fits.open(fname)
    specdata = f[1].data 
    lamb = 10**specdata['loglam'] * u.AA 
    flux = specdata['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1') 

    wv=lamb.value
    mask_left=(wv>x_left1) & (wv<x_left2)
    mask_right=(wv>x_right1) & (wv<x_right2)
    F_left = np.mean(flux[mask_left])
    F_right = np.mean(flux[mask_right])
    wv_left=np.median(wv[mask_left])
    wv_right=np.median(wv[mask_right])

    slope= (F_right-F_left)/(wv_right-wv_left)
    continuum = slope*(wv-wv_left) + F_left

    cont_norm_spec=flux/continuum
    
    eq_est_mask=(wv>x_left2) & (wv<x_right1)
    spec_norm = Spectrum1D(spectral_axis=lamb[eq_est_mask], flux=cont_norm_spec[eq_est_mask]) 
    eq=equivalent_width(spec_norm, regions=SpectralRegion(x_left2 * u.AA, x_right1 * u.AA)) 

    if make_plots:
      fig=px.line(x=wv, y=flux,labels={"x": f"Wavelength [{lamb.unit.to_string()}]","y": f"Flux [{flux.unit.to_string()}]"}, width=1000, height=300)
      fig.update_layout(
          margin=dict(l=0, r=5, t=0, b=0),
          paper_bgcolor="LightSteelBlue",
      )
      fig.add_vrect(x0=x_left1, x1=x_left2, fillcolor="green", opacity=0.25, line_width=0)
      fig.add_vrect(x0=x_right1, x1=x_right2, fillcolor="green", opacity=0.25, line_width=0)
      fig.add_scatter(x=wv,y=continuum,mode='lines')
      display(fig)

      fig_line=px.line(x=wv[eq_est_mask], y=cont_norm_spec[eq_est_mask],labels={"x": f"Wavelength [{lamb.unit.to_string()}]","y": f"Flux [{flux.unit.to_string()}]"}, width=1000, height=300)
      fig_line.update_layout(
          margin=dict(l=0, r=5, t=0, b=0),
          paper_bgcolor="LightSteelBlue",
      )
      fig_line.add_vrect(x0=line-eq.value/2, x1=line+eq.value/2, fillcolor="red", opacity=0.25, line_width=0)
      display(fig_line)
    
    return eq
  else:
    raise ValueError(f"The template {fname} does't exist")

In [ ]:
# Use the above funvtion to estimate equivalent width on the spectrum in the  'F8_-0.5_Dwarf.fits' file.
# look again at the 'ls' comand above to see what data you have 
estimate_eq("F","8","-1.5",6471,6500,6624,6650,6564.75)

In [ ]:
# Try another line...
estimate_eq("F","8","-1.5",3915,3920,3945,3950,3935)

In [ ]:
# Lets make life easier... one can repeat above commands for 10 spectral types 10 subclasses and some metalicity values..
# But we can do this with an automated loop.
# We first define some tables... 
# The tables below might contain combinations of values that are not in the data e.g. B stars contain no metalicity and files are names as 'B0.fits	'

spectral_types=["O","B","A","F","G","K"]#,"M"]
sub_spectral_types=["0","1","2","3","4","5","6","7","8","9"]
metalicity="+0.0"#,"-1.5","-1.0","-0.5","+0.0","+0.5","+1.0"]


# if you are interested in a subset of data you can do it... simply define only G stars for example
# spectral_types=["G"]

df = pd.DataFrame(columns=['SpectralType','Metalicity','Ha_Width','CaK_Width'])
for st in spectral_types:
  for sst in sub_spectral_types:
    try: # Check if file existss
    # See the make_plots=False flag, try 'True' flag to create all plots... this will be slower
      eqHa=estimate_eq(st,sst,metalicity,6471,6500,6624,6650,6564,make_plots=False).value
      eqCaK=estimate_eq(st,sst,metalicity,3915,3920,3945,3950,3935,make_plots=False).value
      
      entry = pd.DataFrame.from_dict({
        "SpectralType": [f"{st}{sst}"],
        "Metalicity":  [metalicity],
        "Ha_Width":eqHa,
        "CaK_Width":eqCaK
        })
    except:
      entry = pd.DataFrame.from_dict({
        "SpectralType": [f"{st}{sst}"],
        "Metalicity":  [metalicity],
        "Ha_Width":np.NaN,
        "CaK_Width":np.NaN
        })
    df = pd.concat([df, entry], ignore_index=True)

In [ ]:
# lets print the df dataframe that has a list of all combinations of  spectral_types subclass and metalicity
# This does not work with O-B type stars that is why you will see that the code has computed nothing  'NaN'
# For extra credit (it is not Mandatory) try to plot a B or O type star and calculate equivalent width

df

In [ ]:
df.plot.bar("SpectralType","Ha_Width",figsize=(16,10))

In [ ]:
df.plot.bar("SpectralType","CaK_Width",figsize=(16,10))